# Getting geographic information for Langtjern and Birkenes
The purpose of this notebook is to  a database with geographic information for the Norwegian basins that are going to be used as a part of the CatchCan project.
The starting point for the analysis are the coordinates for the basin, which will be used to delineate the basin using a high resolution 1x1m digital elevation model (DEM) from kartverket: høydedata.no

The coordinates for the outlets are: 


In [1]:
#The id is the aquamonitor id but it could be any number as long as they are unique
stations = {'Birkenes' : {'lat': 6491473,
                         'long': 105229,
                         'epsg': 25833,
                          'station_id' : 108
                         },
           'Langtjern' : {'lat': 60.37271, #From NVE data
                         'long': 9.72746,
                         'epsg': 4326,
                         'station_id' : 221
                         }
           }

## Loading required packages and getting credentials to connect to the database
This loads the python packages required to process the basins as well as the credentials to different virtual machines where the processing will be performed.
The credentials are stored in mobiserver.no and obtaining them will fail if that server is not running.
The database and the tools necessary to delineate the basins are store in a virtual machine called dtm10. The database is called geonorway.

In [2]:
import requests
import json
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from fabric import Connection
import psycopg2
import gmaps
import pandas.io.sql as sqlio
import getpass
from io import StringIO
import paramiko
import sys
from time import sleep
from osgeo import gdal
import numpy as np
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
import re
import wget
import zipfile

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
cloudKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
gmapsKey = query('''select niva.getToken('gmaps','{}');'''.format(cloudKey)).iloc[0,0]
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(cloudKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(cloudKey)).iloc[0,0])
gmaps.configure(gmapsKey)
not_really_a_file = StringIO(sshKey)
private_key = paramiko.RSAKey.from_private_key(not_really_a_file)
del key,sshKey

mobiserver password:  ·······
vault password:  ···············


## Starting the instance containing the database using the google compute engine api

In [3]:
#Check status of instance
properties = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceType' : "n1-standard-4",
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

cloud = gce(properties, cloudKey)
del cloudKey

#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
          
geonorway = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
    
display("IP of the instance containing the database {}:".format(geonorway))

#The config below can be used to connect to the instance using fabric's Connection
config =  {'host' : geonorway, 'user': 'jose-luis', 'connect_kwargs': {'pkey': private_key } }

'RUNNING'

'IP of the instance containing the database 35.228.235.237:'

## Creating schema to store data

All the geographic data will be stored in the geonarway database, which is running in the virtual machine called dtm10.
IMPORTANT: if you run the cell below, everything in the schema will be erased if it already exist. Make sure you actual want to erase everything and start from scratch.

In [4]:
schema = 'catchcan'

#Helper functio to query the geonorway database. Returns a pandas dataframe with the results of the query.
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result
        
query('''drop schema if exists {0} cascade;
create schema {0};'''.format(schema),fetch=False)       


## Creating table to store the station coordinates

In [5]:
sql = '''drop table if exists {0}.stations;
create table {0}.stations (sid int generated always as identity,
stationName varchar(100) not null,
stationID varchar(50) unique not null,
latitude double precision not null,
longitude double precision not null,
epsg int,
unique(sid),
primary key (sid)
);

insert into {0}.stations (stationName,stationId,latitude,longitude,epsg) values {1};

create index {0}_stations_station_id_idx on {0}.stations(stationId);
alter table {0}.stations add column geom geometry(point,25833);
update {0}.stations
set geom = st_transform(st_setsrid(st_makepoint(longitude,latitude),epsg),25833);

update {0}.stations
set longitude = st_x(geom),
latitude = st_y(geom);

select * from {0}.stations;
'''.format(schema,','.join(['''('{0}',{station_id},{lat},{long},{epsg})'''.format(i,**stations[i])  for i in stations.keys()]))

a = query(sql)
a

,sid,stationname,stationid,latitude,longitude,epsg,geom
0,1,Birkenes,108,6.491473e+06,105229.000000,25833,0101000020E964000000000000D0B0F9400000004054C3...
1,2,Langtjern,221,6.704554e+06,209434.237831,4326,0101000020E9640000F0BE13E7D19009412438A57C6A93...


In [6]:
#Visualizing uploaded stations
fig = gmaps.figure(map_type="TERRAIN")

b = query('''SELECT a.stationname, st_x(st_transform(a.geom,4326)),
    st_y(st_transform(a.geom,4326)), a.stationid
    FROM {0}.stations a; 
    '''.format(schema))


outlets = [{"name": i[0], "id": i[3]} for j,i in b.iterrows()]
locations = [(float(i[2]),float(i[1])) for j,i in b.iterrows()]
info_box_template = """
<dl>
<font color="black">
<dt>Name</dt><dd>{name}</dd>
<dt>Id</dt><dd>{id}</sup></dd>
</font>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Computing the basins using the 10m dem

### First finding the point in the flow accumulation raster closest to the given coordinates
A common problem with coordinates is that they do fall on an actual "river". In order for the basin delineation algorithm to work, the given coordinates should fall inside a cell considered a river in the flow accumulation raster. 
In order to guarantee this we will nudge, if necessary, the given coordinates so they fall in a cell considered to be a river. The process is not fool proof and the results should be quality controlled. 

In [7]:
#Please note that in the outlet table we will setup the srid of the outlet to 3035 since
#that is the epsg of the rasters we use to compute the basins
sql = '''drop table if exists {schema}.outlet cascade;
create table {schema}.outlet as select sid, stationname, stationid, latitude, longitude, st_transform(geom,25833) as geom from {schema}.stations as a;
update {schema}.outlet
set latitude=st_y(geom),
longitude=st_x(geom);

create index {schema}_outlet_idx on {schema}.outlet using gist(geom);
alter table {schema}.outlet add primary key (sid);
alter table {schema}.outlet add constraint fk_basin
foreign key(sid) 
references {schema}.stations(sid);

'''.format(schema=schema)
query(sql,fetch=False)
display(query('''select * from {schema}.outlet order by latitude desc;'''.format(schema=schema)))

,sid,stationname,stationid,latitude,longitude,geom
0,2,Langtjern,221,6.704554e+06,209434.237831,0101000020E9640000F0BE13E7D19009412438A57C6A93...
1,1,Birkenes,108,6.491473e+06,105229.000000,0101000020E964000000000000D0B0F9400000004054C3...


In [8]:
#Moving outlet to the closest river, within a 500m radius
sql = '''update {schema}.outlet as a 
set geom = 
case 
when st_intersects(st_buffer(a.geom,500),b.geom) then
(
select st_closestpoint(st_union(b.geom),a.geom) from elv.elvenett25833 as b 
where st_intersects(st_buffer(a.geom,500),b.geom)
)
else a.geom
end
from elv.elvenett25833 as b where st_intersects(st_buffer(a.geom,500),b.geom);
;
update {schema}.outlet
set latitude=st_y(geom),
longitude=st_x(geom);

'''.format(schema=schema)
query(sql,fetch=False)

In [9]:
#Loading the flow accumulation raster locally. We will do the computation one basin at a time. We exclude the stations in Svalbard
#for which we have no elevation data
sql = '''with bla as
(select sid,stationid,stationname,st_transform(geom,3045) as geom from {schema}.stations)
select distinct a.gid, array_agg(sid) as sids
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom)
group by a.gid;'''.format(schema = schema)

a = query(sql)
display(a)

,gid,sids
0,12,[2]
1,20,[1]


In [10]:
#Processing all stations within one basin
import numpy as np
from tqdm import tqdm

#Creating temporary directory to download data and place intermediary files
with Connection('localhost') as c:
    c.local('rm -rf geodata && mkdir geodata')

#A river defined as a pixel with more than 1km2  upstream areas (10k cells)
def find_nearest_river(pixel):
    #Returns index to numpy array
    distances = (nonzero[:,0] - pixel[0]) ** 2 + (nonzero[:,1] - pixel[1]) ** 2
    nearest_index = np.argmin(distances)
#     display(nearest_index)
    return nonzero[nearest_index]

def coordsFromPixel(i,j):
    #i, j are pixel coordinated in the numpy array
    long = xoffset + (j+0.5) * px_w  #adding half a pixel width so the coordinates are centered
    lat = yoffset + (i+0.5) * px_h  
    return long,lat

def pixelFromCoord(long,lat):
    #Returns index to numpy array
    i = (long - xoffset) / px_w
    j = (lat - yoffset) / px_h
#     display([sid,long,lat,i,j])
    return (int(j),int(i))

sql = '''select sid, longitude, latitude from {schema}.outlet where sid in ({idList}); '''

updateSql = '''update {schema}.outlet
set longitude = {long},
latitude = {lat}
where sid = {sid};

update {schema}.outlet
set geom = st_setsrid(st_makepoint(longitude,latitude),25833)
where sid={sid};
'''

for i,j in tqdm(a.iterrows()):
    gid = j['gid']
    sids = j['sids']
#     display(gid,sids)    
    coords = query(sql.format(schema=schema,idList=str(sids)[1:-1]))
    coords.set_index('sid',inplace=True)
#     display(coords)
    with Connection(**config) as c:
        c.get('/home/jose-luis/flatLake/basin_{}_flow_acc.tif'.format(gid),'./geodata/')
        dataset = gdal.Open('geodata/basin_{}_flow_acc.tif'.format(gid),gdal.GA_ReadOnly)
        xoffset, px_w, rot1, yoffset, rot2, px_h = dataset.GetGeoTransform()
#         display([xoffset, px_w, rot1, yoffset, rot2, px_h])
        #IMPORTANT: the index order is inverted when loading into a numpy array. 
        #This is the reason of the index gymnastics in the following code
        river = np.array(dataset.GetRasterBand(1).ReadAsArray())
        nonzero = np.argwhere(river > 2000)
#         display(len(nonzero))
        for sid in sids:
            long = coords.loc[sid,'longitude']
            lat = coords.loc[sid,'latitude']
            currentPixel = pixelFromCoord(long,lat)
#             display("Current flow_acc: {}".format(river[currentPixel[1],currentPixel[0]]))
            riverPixel = find_nearest_river(currentPixel)
#             display("Updated flow_acc: {}".format(river[riverPixel[0],riverPixel[1]]))
#             display([long,lat,currentPixel,riverPixel])
            new_x,new_y = coordsFromPixel(riverPixel[0],riverPixel[1])
#             display([sid,long,lat,new_x,new_y])
            query(updateSql.format(schema=schema,sid=sid,long=new_x,lat=new_y),fetch=False)
    with Connection('localhost') as c:
        c.local('rm geodata/basin_{}_flow_acc.tif'.format(gid))
    
#             display([long,lat,new_x,new_y])

2it [00:13,  6.96s/it]


In [11]:
#Finding out the gid number for the flow accumulation raster
sql = '''with bla as
(select sid,stationid,st_transform(geom,3045) as geom from {schema}.stations)
select b.sid,b.stationid,a.gid,a.vassomr,st_area(a.geom)/1e6 as area
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom);'''.format(schema=schema)

a = query(sql)
display(a)

query('drop table if exists {schema}.basins;'.format(schema=schema),fetch=False)
query('''create table {schema}.basins(sid integer not  null,area double precision, geom geometry(multipolygon,25833),
primary key (sid),
constraint fk_outlet
foreign key(sid) 
references {schema}.outlet(sid)
);
'''.format(schema=schema),fetch=False)

#Creating folder to store results
with Connection(**config) as c:
    c.run('rm -rf watershed && mkdir watershed')
    

script='''#! /bin/bash
cd watershed
rm -f out.* basin.*

pgsql2shp -f out{sid}.shp geonorway "select sid as id, geom from {schema}.outlet where sid={sid}"
mpiexec -n 4 gagewatershed -p /home/jose-luis/flatLake/basin_{gid}_flow_dir.tif -o out{sid}.shp -gw watershed_{sid}.tif
gdal_polygonize.py -8 -f "ESRI Shapefile" watershed_{sid}.tif basin{sid}.shp
shp2pgsql -d -s 25833 basin{sid}.shp {schema}.temp | psql -d geonorway
echo "insert into {schema}.basins(sid,geom) select dn, st_makevalid(geom) from {schema}.temp;" | psql -d geonorway
'''

for i,j in a.iterrows():   
# for sid in sids:
#     sid=j['station_id']
    display([j['sid'],j['stationid']])
    with Connection(**config) as c:
        with open('script.sh','w') as f:
            cond = {'sid': j['sid'], 'gid': j['gid'], 'schema': schema}
            f.write(script.format(**cond))
        c.put('script.sh')
        c.run('chmod +x script.sh')
        c.run('./script.sh',hide='both')
    

query('update {schema}.basins set area=st_area(geom)'.format(schema=schema),fetch=False);
query('create index {schema}_basins_idx on {schema}.basins using gist(geom);'.format(schema=schema),fetch=False)

,sid,stationid,gid,vassomr,area
0,1,108,20,Tovdalsvassdraget/Lillesand kommune,2406.825233
1,2,221,12,Drammensvassdraget/Drammensfjorden vest,17212.146367


[1, '108']

[2, '221']

In [12]:
a = query('''SELECT json_build_object('type', 'FeatureCollection',
                                      'features', json_agg(json_build_object(
                                                                'type',       'Feature',
                                                                'label',      b.stationname,
                                                                'geometry',   ST_AsGeoJSON(ST_ForceRHR(St_Transform(a.geom,4326)))::json,
                                                                'properties', jsonb_set(row_to_json(a)::jsonb,'{{a.geom}}','0',false)
                                                                )
                                                            )
                                     )
             FROM {0}.basins as a join {0}.outlet as b on b.sid=a.sid;'''.format(schema))

fig = gmaps.figure(map_type="TERRAIN")
fig.add_layer(gmaps.geojson_layer(a.iloc[0,0]))

b = query('''SELECT a.stationname, st_x(st_transform(a.geom,4326)),
    st_y(st_transform(a.geom,4326)), c.area/1e6 as area, a.sid
    FROM {0}.outlet AS a
    JOIN {0}.basins as c
    on a.sid=c.sid
    '''.format(schema))


outlets = [{"name": i[0], "area": i[3], "id": i[4]} for j,i in b.iterrows()]
locations = [(float(i[2]),float(i[1])) for j,i in b.iterrows()]
info_box_template = """
<dl>
<font color="black">
<dt>Name</dt><dd>{name}</dd>
<dt>Id</dt><dd>{id}</sup></dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</font>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

## Getting 1m elevation data from the 1x1m DEM


In [13]:
#The 1m data cannot be downloaded all at once. It is split in groups. First we find out which groups need to 
#downloaded
intersection =  query( '''select distinct b.sid,a.gruppe from nhm.gruppering as a,
{schema}.basins as b
where st_intersects(st_buffer(a.geom,1000),b.geom)'''.format(schema=schema))
intersection

,sid,gruppe
0,1,11-10
1,2,1213-11


In [14]:
#Figuring out the link of the group that needs to be dowloaded using bs4
baseURL = 'https://hoydedata.no/LaserInnsyn/'

def getSoup(url,re_str):
    s=requests.Session()
    only_a_tags = SoupStrainer("a", href=True)
    request=s.get(url)
    soup = BeautifulSoup(request.text,'lxml')
    links = {i.a.text : (urljoin(baseURL,i.a['href']),int(i.find_all('td')[1].text.replace(u'\xa0MB', u' ')))  
            for i in soup.find_all('tr',{'class':"dlContent sone33"})
            if i.a and re_str in i.a.text}
    return links

dtm1_links = getSoup(baseURL,'DTM1 ')
display(dtm1_links,'',
        'The total size of the available dtm data is {} GB'.format(sum([value[1] for dummy,value in dtm1_links.items()])/1000)
       )

{'DTM1 10-10': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/29',
  36233),
 'DTM1 11-10': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/33',
  56086),
 'DTM1 12-10': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/38',
  10680),
 'DTM1 10-11': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/30',
  47830),
 'DTM1 11-11': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/34',
  77848),
 'DTM1 1213-11': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/40',
  91526),
 'DTM1 10-12': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/31',
  54112),
 'DTM1 11-12': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/35',
  70487),
 'DTM1 1213-12': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/41',
  101642),
 'DTM1 10-13': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/32',
  18379),
 'DTM1 11-13': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/36',
  67395),
 'DTM1 1213-13': ('https://hoydedata.no/LaserInnsyn/Home/DownloadFile/42',
  92861

''

'The total size of the available dtm data is 1299.928 GB'

In [15]:
#The groups that need to be downloaded are:

download_links = [value for key, value in dtm1_links.items() if any(x in key for x in list(intersection.gruppe))]
#Performing download. This takes a while, check that it hasn't been downloaded before.
if False: #Change to true if not already downloaded
    for i in download_links:
        display('''Downloading {} with size {} MB'''.format(i[0],i[1]))
        filename = '/home/jovyan/shared/' + i[0].split('/')[-1] + '.zip'
        wget.download(i[0],filename)
        with zipfile.ZipFile(filename,"r") as zip_ref:
            zip_ref.extractall("/home/jovyan/shared/geodata/")

In [16]:
#Now getting the actual tiles that cover the basins
a = query('''select a.sid,array_agg('/home/jovyan/shared/geodata/' || b.name || '.tif') as tilelist from nhm.kartblad as b,
{schema}.basins as a
where st_intersects(a.geom,st_buffer(b.geom,1000))
group by a.sid;'''.format(schema=schema))
a.set_index("sid",inplace=True)
a

,tilelist
sid,
1,[/home/jovyan/shared/geodata/33-113-104.tif]
2,[/home/jovyan/shared/geodata/33-120-118.tif]


In [17]:
#Merging all the tiles and masking them using the buffered 10m basins
for i in a.index:
    tiles = a.loc[i]['tilelist']
    with Connection('localhost') as c:
        #If there are too many tiles this command might fail
        #The nodata value should eventually be obtained programatically
        c.local('''gdal_merge.py -o {} -n -32767 {}'''.format('{}.tif'.format(i),' '.join(tiles)))
        c.local('''gdalwarp -s_srs EPSG:25833 -t_srs EPSG:25833 -srcnodata -32767 -dstnodata -32767 -cutline PG:"dbname=geonorway host={ip} user='jose-luis'" -csql 'select st_buffer(geom,1000) from {schema}.basins where sid = {sid}' -crop_to_cutline -of GTiff -overwrite {sid}.tif {sid}_cropped.tif'''.format(schema=schema,sid=i,ip=config['host']),replace_env=False)   

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2840P x 2720L.
Processing 1.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4330P x 5740L.
Processing 2.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


In [48]:
#Preprocessing the 1m elevation dataset

#Adding column with buffered basins so we can add an index to the buffer in order to make the queries using it faster
sql = '''alter table catchcan.basins add column if not exists buffer geometry(polygon,25833);
update catchcan.basins set buffer = st_buffer(geom,1000);
create index if not exists catchcan_basins_buffer_idx on catchcan.basins using gist(buffer);'''
query(sql,fetch=False)
    
#Burning in the river into the raster
#Query to select only rivers within the buffer basin
sql = '''select st_union(st_intersection(a.geom,b.buffer)) as geom from elv.elvenett25833 as a, {schema}.basins as b where st_intersects(b.buffer,a.geom) and b.sid={sid}'''

for i in [1,2]:
    with Connection('localhost') as c:
        c.local('''cp {sid}_cropped.tif {sid}_rivers.tif && gdal_rasterize -b 1 -burn -20 -add -sql '{sql}'  PG:"dbname=geonorway host={ip} user='jose-luis'"  {sid}_rivers.tif '''.format(
            sql=sql.format(schema=schema,sid=i),
            sid=i,
            ip=config['host'] )
               )

#Creating folder to store results and putting the elevation dataset in the vm
with Connection(**config) as c:
    c.run('rm -rf watershed && mkdir watershed')
    for i in [1,2]:
        c.put('{sid}_rivers.tif'.format(sid=i),'watershed/{sid}_rivers.tif'.format(sid=i))
 
    
#Preprocessing the DEM
script='''#! /bin/bash
cd watershed
rm -f out.* basin.*

mpiexec -n 4 pitremove -z {sid}_rivers.tif -fel {sid}_filled.tif 
mpiexec -n 4 d8flowdir -fel {sid}_filled.tif -p {sid}_flow_dir.tif
mpiexec -n 4 aread8 -p {sid}_flow_dir.tif -nc -ad8 {sid}_flow_acc.tif
'''

for i in [1,2]:   
# for sid in sids:
#     sid=j['station_id']
#     display([j['sid'],j['stationid']])
    with Connection(**config) as c:
        with open('script.sh','w') as f:
            cond = {'sid': i, 'schema': schema, 'gid' : i}
            f.write(script.format(**cond))
        c.put('script.sh')
        c.run('chmod +x script.sh')
        c.run('./script.sh',hide='both')
    

# query('update {schema}.basins set area=st_area(geom)'.format(schema=schema),fetch=False);
# query('create index {schema}_basins_idx on {schema}.basins using gist(geom);'.format(schema=schema),fetch=False)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [49]:
#Moving the outlet to fall on the flow accumulation raster
sql = '''drop table if exists {schema}.outlet_1m;
create table {schema}.outlet_1m as select * from {schema}.outlet;
select * from {schema}.outlet;'''
a = query(sql.format(schema=schema))

updateSql = '''update {schema}.outlet_1m
set longitude = {long},
latitude = {lat}
where sid = {sid};

update {schema}.outlet_1m
set geom = st_setsrid(st_makepoint(longitude,latitude),25833)
where sid={sid};
'''

for i,j in a.iterrows():
#     display(coords)
    with Connection(**config) as c:
        c.get('/home/jose-luis/watershed/{}_flow_acc.tif'.format(j['sid']),'./geodata/')
        dataset = gdal.Open('geodata/{}_flow_acc.tif'.format(j['sid']),gdal.GA_ReadOnly)
        xoffset, px_w, rot1, yoffset, rot2, px_h = dataset.GetGeoTransform()
#         display([xoffset, px_w, rot1, yoffset, rot2, px_h])
        #IMPORTANT: the index order is inverted when loading into a numpy array. 
        #This is the reason of the index gymnastics in the following code
        river = np.array(dataset.GetRasterBand(1).ReadAsArray())
        nonzero = np.argwhere(river > 10000)
#         display(len(nonzero))
        
        long = j['longitude']
        lat = j['latitude']
        currentPixel = pixelFromCoord(long,lat)
#             display("Current flow_acc: {}".format(river[currentPixel[1],currentPixel[0]]))
        riverPixel = find_nearest_river(currentPixel)
#             display("Updated flow_acc: {}".format(river[riverPixel[0],riverPixel[1]]))
#             display([long,lat,currentPixel,riverPixel])
        new_x,new_y = coordsFromPixel(riverPixel[0],riverPixel[1])
#             display([sid,long,lat,new_x,new_y])
        query(updateSql.format(schema=schema,sid=j['sid'],long=new_x,lat=new_y),fetch=False)
    with Connection('localhost') as c:
        c.local('rm geodata/{}_flow_acc.tif'.format(j['sid']))

In [50]:
#Computing the basins from the 1m DEM
query('drop table if exists {schema}.basinsdtm1;'.format(schema=schema),fetch=False)
query('''create table {schema}.basinsdtm1(sid integer not  null,area double precision, geom geometry(multipolygon,25833),
primary key (sid),
constraint fk_outlet
foreign key(sid) 
references {schema}.outlet(sid)
);
'''.format(schema=schema),fetch=False)

script='''pgsql2shp -f out{sid}.shp geonorway "select sid as id, geom from {schema}.outlet_1m where sid={sid}"
mpiexec -n 4 gagewatershed -p /home/jose-luis/watershed/{sid}_flow_dir.tif -o out{sid}.shp -gw watershed_{sid}.tif
gdal_polygonize.py -8 -f "ESRI Shapefile" watershed_{sid}.tif basin{sid}.shp
shp2pgsql -d -s 25833 basin{sid}.shp {schema}.temp | psql -d geonorway
echo "insert into {schema}.basinsdtm1(sid,geom) select dn, st_makevalid(geom) from {schema}.temp;" | psql -d geonorway
'''

for i in a['sid'].values:   
# for sid in sids:
#     sid=j['station_id']
#     display([j['sid'],j['stationid']])
    with Connection(**config) as c:
        with open('script.sh','w') as f:
            cond = {'sid': i, 'schema': schema}
            f.write(script.format(**cond))
        c.put('script.sh')
        c.run('chmod +x script.sh')
        c.run('./script.sh',hide='both')
        
query('update {schema}.basinsdtm1 set area=st_area(geom)'.format(schema=schema),fetch=False);
query('create index {schema}_basinsdtm1_idx on {schema}.basins using gist(geom);'.format(schema=schema),fetch=False)

In [51]:
a = query('''SELECT json_build_object('type', 'FeatureCollection',
                                      'features', json_agg(json_build_object(
                                                                'type',       'Feature',
                                                                'label',      b.stationname,
                                                                'geometry',   ST_AsGeoJSON(ST_ForceRHR(St_Transform(a.geom,4326)))::json,
                                                                'properties', jsonb_set(row_to_json(a)::jsonb,'{{a.geom}}','0',false)
                                                                )
                                                            )
                                     )
             FROM {0}.basinsdtm1 as a join {0}.outlet_1m as b on b.sid=a.sid;'''.format(schema))

fig = gmaps.figure(map_type="TERRAIN")
fig.add_layer(gmaps.geojson_layer(a.iloc[0,0]))

b = query('''SELECT a.stationname, st_x(st_transform(a.geom,4326)),
    st_y(st_transform(a.geom,4326)), c.area/1e6 as area, a.sid
    FROM {0}.outlet_1m AS a
    JOIN {0}.basinsdtm1 as c
    on a.sid=c.sid
    '''.format(schema))


outlets = [{"name": i[0], "area": i[3], "id": i[4]} for j,i in b.iterrows()]
locations = [(float(i[2]),float(i[1])) for j,i in b.iterrows()]
info_box_template = """
<dl>
<font color="black">
<dt>Name</dt><dd>{name}</dd>
<dt>Id</dt><dd>{id}</sup></dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</font>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))